In [1]:
!pip install lightning
!pip install polars

!pip install --extra-index-url https://pypi.nvidia.com --upgrade nvidia-dali-cuda120
!pip install --extra-index-url https://pypi.nvidia.com --upgrade nvidia-dali-tf-plugin-cuda120

!pip install git+https://github.com/kacky355/my_libraries.git


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.1/54.1 kB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 33.7 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://pypi.nvidia.com
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 301.6/301.6 MB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.0/13.0 MB 87.3 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://pypi.nvidia.com
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 455.8/455.8 kB 9.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... - done
  Created wheel for nvidia-dali-tf-plugin-cuda120: filename=nvidia_dali_tf_plugin_cuda120-1.39.0-cp310-cp310-linux_x86_64.whl size=128314 sha256=75756fbbbedafbb84e05fd2cf0ccee954ae5ebbdb5822d58dafe81ab54628a59
  Stored in directory: /root/.cache/pip/wheels/41/d2/6e/deb92fca17a5dff9c8146227978214fc8eedd3bbff0b345695
Successfully built nvidia-dali-tf-plugin-cuda120
  Cloning https://github.com/kacky355/my_libraries.

In [2]:
import random
import os
import glob

import matplotlib.pyplot as plt
import gc
from tqdm.notebook import tqdm

import numpy as np
import pandas as pd
import polars as pl

from nvidia.dali import pipeline_def, Pipeline
import nvidia.dali.fn as fn
import nvidia.dali.types as types
import nvidia.dali.tfrecord as tfrec
from nvidia.dali.plugin.pytorch import DALIGenericIterator, LastBatchPolicy

import math
import torch
from torch import nn, Tensor
import torch.nn.functional as F
import torch.optim as optim
from torchmetrics.classification import MultilabelAveragePrecision as mAP
import lightning as L
from lightning.pytorch.callbacks import EarlyStopping, ModelCheckpoint, LearningRateMonitor,TQDMProgressBar

from sklearn.metrics import average_precision_score as APS

import tensorflow as tf

from logger.mylogger import get_my_logger

2024-07-08 07:43:57.341153: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-08 07:43:57.341216: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-08 07:43:57.342879: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [3]:
# class GriffinLM(nn.Module):
#     def __init__(self, config: GriffinConfig):
#         super().__init__()
#         self.embedding = nn.Embedding(config.vocab_size, config.dim)
#         self.backbone = nn.ModuleList([Block(config) for _ in range(config.num_layers)])
#         self.output_norm = RMSNorm(dim=config.dim)
#         self.lm_head = nn.Linear(config.dim, config.vocab_size, bias=False)

#         with torch.no_grad():
#             self.embedding.weight.normal_(std=config.dim**-0.5)
#             self.lm_head.weight.normal_(std=config.dim**-0.5)

#         self.tie_weights_()

#     def tie_weights_(self):
#         self.lm_head.weight = self.embedding.weight

#     def parameter_groups(self, weight_decay=1e-2):
#         return [
#             {'params': self.embedding.parameters(), 'weight_decay': 0.0}, # lm_head is tied here
#             # do not decay biases and single-column parameters (forget_base, rmsnorm), those are usually scales
#             {'params': (p for p in self.backbone.parameters() if p.dim() < 2), 'weight_decay': 0.0},
#             {'params': (p for p in self.backbone.parameters() if p.dim() >= 2), 'weight_decay': weight_decay},
#             {'params': self.output_norm.parameters(), 'weight_decay': 0.0},
#         ]

#     def forward(self, input_ids):
#         x = self.embedding(input_ids)
#         for block in self.backbone:
#             x = block(x)
#         x = self.lm_head(self.output_norm(x))
#         return x


In [4]:
%%writefile config.py
import os
import glob
from dataclasses import dataclass


class CFG:
    DEBUG = False
    MODEL_NAME = 'Transformer'

    EPOCHS = 2
    BATCH_SIZE = 2048
    NBR_FOLDS = 15
    NUM_TRAINS = 91_854_569
    NUM_VALIDS = 6_561_041
    STEPS_PER_EPOCH_TRAIN = (NUM_TRAINS -1) //BATCH_SIZE +1
    STEPS_PER_EPOCH_VALID = (NUM_VALIDS -1) //BATCH_SIZE +1


    SELECTED_FOLDS = [0]

    BASE_DIR = '/kaggle/working/'
    DATA_SOURCE = '/kaggle/input/belka-train-valid-tfrecords-1d-preprocessed'
    DATA_SOURCE_DOWNSAMPLE = '/kaggle/input/be-train-tfrec-downsample'
#     TRAINS = glob.glob('/kaggle/input/be-train-tfrec-downsample/trains/BELKA_train_*.tfrecord')
    TRAINS = glob.glob(os.path.join(DATA_SOURCE, 'train','BELKA_train_*.tfrecord'))
#     TRAINS = glob.glob(os.path.join(DATA_SOURCE_DOWNSAMPLE, 'trains','BELKA_train_*.tfrecord'))
    TRAINS.sort()
    TRAIN_IDX = glob.glob(os.path.join(DATA_SOURCE, 'tf_idx', 'train_*.idx'))
#     TRAIN_IDX = glob.glob(os.path.join(DATA_SOURCE_DOWNSAMPLE, 'tf_idx', 'train_*.idx'))
    TRAIN_IDX.sort()
    VALIDS = glob.glob(os.path.join(DATA_SOURCE, 'valid/*'))
    VALIDS.sort()
    VALID_IDX = glob.glob(os.path.join(DATA_SOURCE, 'tf_idx', 'valid_*.idx'))
    VALID_IDX.sort()
    
    
    TRAINS = TRAINS[:18]
    TRAIN_IDX = TRAIN_IDX[:18]

    SEED = 2024


    FEATURES = [f'enc{i}' for i in range(142)]
    TARGETS = ['bind1', 'bind2', 'bind3']
    COLUMNS = FEATURES + TARGETS

    NUM_CLASSES = 3
    SEQ_LENGTH = 142
    LR = 1e-5
    WD = 0.05


    if DEBUG:
        EPOCHS = 3
        TRAINS = TRAINS[:4]
        TRAIN_IDX = TRAIN_IDX[:4]

@dataclass 
class TransformerConfig:
    batch:int = CFG.BATCH_SIZE
    input_dim: int = CFG.SEQ_LENGTH
    input_dim_embedding: int = 37
    hidden_dim: int = 256
    num_heads: int = 4
    num_layers: int = 3
    dropout: float = 0.2
    out_dim: int = CFG.NUM_CLASSES
    learning_rate:float = CFG.LR
    weight_decay: float = CFG.WD

Writing config.py


In [5]:
from config import CFG
CFG.TRAINS


['/kaggle/input/belka-train-valid-tfrecords-1d-preprocessed/train/BELKA_train_00.tfrecord',
 '/kaggle/input/belka-train-valid-tfrecords-1d-preprocessed/train/BELKA_train_01.tfrecord',
 '/kaggle/input/belka-train-valid-tfrecords-1d-preprocessed/train/BELKA_train_02.tfrecord',
 '/kaggle/input/belka-train-valid-tfrecords-1d-preprocessed/train/BELKA_train_03.tfrecord',
 '/kaggle/input/belka-train-valid-tfrecords-1d-preprocessed/train/BELKA_train_04.tfrecord',
 '/kaggle/input/belka-train-valid-tfrecords-1d-preprocessed/train/BELKA_train_05.tfrecord',
 '/kaggle/input/belka-train-valid-tfrecords-1d-preprocessed/train/BELKA_train_06.tfrecord',
 '/kaggle/input/belka-train-valid-tfrecords-1d-preprocessed/train/BELKA_train_07.tfrecord',
 '/kaggle/input/belka-train-valid-tfrecords-1d-preprocessed/train/BELKA_train_08.tfrecord',
 '/kaggle/input/belka-train-valid-tfrecords-1d-preprocessed/train/BELKA_train_09.tfrecord',
 '/kaggle/input/belka-train-valid-tfrecords-1d-preprocessed/train/BELKA_train_10

In [6]:
%%writefile models.py
from config import CFG, TransformerConfig
import math

import torch
from torch import nn, Tensor
import torch.nn.functional as F
import torch.optim as optim
import lightning as L
from torchmetrics.classification import MultilabelAveragePrecision as mAP

from sklearn.metrics import average_precision_score as APS


class RMSNorm(nn.Module):
    def __init__(self, *, dim):
        super().__init__()
        self.scale = dim**-0.5
        self.gamma = nn.Parameter(torch.ones(dim))

    def forward(self, x):
        x = x / x.norm(p=2, dim=-1, keepdim=True)
        return self.gamma / self.scale * x


class PositionalEncoder(nn.Module):
    def __init__(self, d_model, dropout_p, max_len=5000):
        super(PositionalEncoder, self).__init__()

        self.dropout = nn.Dropout(dropout_p)

        pos_encoding = torch.zeros(max_len, d_model)
        positions_list = torch.arange(0, max_len, dtype= torch.float).unsqueeze(1)
        division_term = torch.exp(torch.arange(0, d_model, 2).float() * (-math.log(10000.0))/d_model)

        pos_encoding[:, 0::2] = torch.sin(positions_list * division_term)
        pos_encoding[:, 1::2] = torch.cos(positions_list * division_term)

        pos_encoding = pos_encoding.unsqueeze(0).transpose(0,1)
        self.register_buffer('pos_encoding', pos_encoding)

    def forward(self, token_embedding: torch.Tensor) -> torch.Tensor:
#         residual conection + pos_encodeing
        return self.dropout(token_embedding + self.pos_encoding[:token_embedding.size(0),:token_embedding.size(1), :])


class Encoder(nn.Module):
    def __init__(self, hidden_dim, num_heads, num_layers, dropout=0.1):
        super().__init__()
        encoder_layer = nn.TransformerEncoderLayer(
            d_model= hidden_dim,
            nhead= num_heads,
            dim_feedforward= hidden_dim*4,
            dropout=dropout,
            activation='gelu',
            batch_first= True,
            norm_first= True,
        )
        self.encoder = nn.TransformerEncoder(encoder_layer, num_layers)

    def forward(self, src, src_key_padding_mask):
        output = self.encoder(src, src_key_padding_mask=src_key_padding_mask)
        return output


    
    

class LMTransformer(L.LightningModule):
    def __init__(self):
        super().__init__()
        self.save_hyperparameters()
        self.val_preds = []
        self.val_y = []
        self.metrics_map = mAP(3,thresholds=None, average='micro')
        self.metrics_ap = mAP(3,thresholds=None, average='none')
        self.config = TransformerConfig()
        self.embedding = nn.Embedding(num_embeddings= self.config.input_dim_embedding, 
                                      embedding_dim= self.config.hidden_dim, 
                                      padding_idx= 0)
        self.norm1 = nn.BatchNorm1d(self.config.hidden_dim)
        self.cls_token = nn.Parameter(torch.zeros(1,1,self.config.hidden_dim))
        self.positional_encoder = PositionalEncoder(d_model= self.config.hidden_dim, 
                                                    dropout_p= self.config.dropout, 
                                                    max_len = self.config.batch)
        self.encoder = Encoder(hidden_dim= self.config.hidden_dim, 
                               num_heads= self.config.num_heads, 
                               num_layers= self.config.num_layers, 
                               dropout= self.config.dropout)
#         self.norm2 = RMSNorm(dim=self.config.hidden_dim)

        self.mlp_head = nn.Sequential(
            nn.LayerNorm(self.config.hidden_dim),
            nn.Linear(self.config.hidden_dim,1024),
            nn.GELU(),
            nn.Dropout(self.config.dropout),
            nn.Linear(1024, 512),
            nn.GELU(),
            nn.Dropout(self.config.dropout),
            nn.Linear(512,self.config.out_dim)
            
        )


    def forward(self, x, mask):
        x = self.embedding(x)
        x = self.norm1(x.permute(0,2,1))
        x = x.permute(0,2,1)
        cls_tokens = self.cls_token.repeat_interleave(x.shape[0], dim= 0).clone()
#         cls_tokens = self.cls_token.clone().repeat(x.shape[0], 1, 1)

        x = torch.cat((cls_tokens, x), dim= 1)
        x = self.positional_encoder(x)
        x = self.encoder(x, src_key_padding_mask=mask)
#         x = self.norm2(x)
        x = self.mlp_head(x[:,0,:])        
        return x



    def training_step(self, batch, batch_idx):
        x, mask, y = self.process_batch(batch)
        logits = self(x, mask)
        preds = torch.sigmoid(logits)
        loss = F.binary_cross_entropy_with_logits(logits, y)
        self.log('train_loss', loss, prog_bar=True, sync_dist=True)
        return loss


    def validation_step(self, batch, batch_idx):
        x, mask, y = self.process_batch(batch)
        logits = self(x, mask)
        preds = torch.sigmoid(logits)
        loss = F.binary_cross_entropy_with_logits(logits, y)
        self.log('val_loss', loss, prog_bar=True, sync_dist=True)
        self.val_preds.append(preds)
        self.val_y.append(y)
        return loss

    def on_validation_epoch_end(self):
        preds = torch.cat(self.val_preds, 0)
        y = torch.cat(self.val_y, 0).to(torch.int)
        APs = self.metrics_ap(preds, y)
        for i in range(3):
            self.log(f'val_AP_bind{i}',APs[i], sync_dist=True)
        meanAP = self.metrics_map(preds, y)
        self.log('val_mAP', meanAP, prog_bar=True, sync_dist=True)
        self.val_preds.clear()
        self.val_y.clear()


    def test_step(self, batch, batch_idx):
        x, mask, y = self.process_batch(batch)
        logits = self(x, mask)
        loss = F.binary_cross_entropy_with_logits(logits, y)
        self.log('test_loss', loss)
        return loss


    def configure_optimizers(self):
        optimizer = torch.optim.AdamW(self.parameters(), lr=self.config.learning_rate, weight_decay=self.config.weight_decay)
        return optimizer

    def process_batch(self, batch):
        X, y = batch[0].clone().long(),  batch[1].clone()
        mask = (X != 0).clone().long()
        mask = torch.cat((torch.ones(X.shape[0],1).to(0), mask), dim=1)
        
        return X, mask, y




class DemoModel(L.LightningModule):
    def __init__(self, input_dim=142, input_dim_embedding=37, hidden_dim=128, num_filters=32, output_dim=3, lr=1e-3, weight_decay=1e-6):
        super().__init__()
        self.save_hyperparameters()
        self.val_preds = []
        self.val_y = []
        self.metrics_map = mAP(3,thresholds=None, average='micro')
        self.metrics_ap = mAP(3,thresholds=None, average='none')

        self.embedding = nn.Embedding(num_embeddings=self.hparams.input_dim_embedding, embedding_dim=self.hparams.hidden_dim, padding_idx=0)
        self.conv1 = nn.Conv1d(in_channels=self.hparams.hidden_dim, out_channels=self.hparams.num_filters, kernel_size=3, stride=1, padding=0)
        self.conv2 = nn.Conv1d(in_channels=self.hparams.num_filters, out_channels=self.hparams.num_filters*2, kernel_size=3, stride=1, padding=0)
        self.conv3 = nn.Conv1d(in_channels=self.hparams.num_filters*2, out_channels=self.hparams.num_filters*3, kernel_size=3, stride=1, padding=0)
        self.global_max_pool = nn.AdaptiveMaxPool1d(1)
        self.fc1 = nn.Linear(self.hparams.num_filters*3, 1024)
        self.dropout = nn.Dropout(0.1)
        self.fc2 = nn.Linear(1024, 1024)
        self.fc3 = nn.Linear(1024, 512)
        self.output = nn.Linear(512, self.hparams.output_dim)

    def forward(self, x):
        x = self.embedding(x).permute(0,2,1)
        x = F.relu(self.conv1(x))
        x = F.relu(self.conv2(x))
        x = F.relu(self.conv3(x))
        x = self.global_max_pool(x).squeeze(2)
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = F.relu(self.fc2(x))
        x = self.dropout(x)
        x = F.relu(self.fc3(x))
        x = self.dropout(x)
        x = self.output(x)
        return x

    def training_step(self, batch, batch_idx):
        x, y = self.process_batch(batch)
        logits = self(x)
        preds = torch.sigmoid(logits)
        loss = F.binary_cross_entropy_with_logits(logits, y)
        self.log('train_loss', loss, prog_bar=True, sync_dist=True)
        return loss


    def validation_step(self, batch, batch_idx):
        x, y = self.process_batch(batch)
        logits = self(x)
        preds = torch.sigmoid(logits)
        loss = F.binary_cross_entropy_with_logits(logits, y)
        self.log('val_loss', loss, prog_bar=True, sync_dist=True)
        self.val_preds.append(preds)
        self.val_y.append(y)
        return loss

    def on_validation_epoch_end(self):
        preds = torch.cat(self.val_preds, 0)
        y = torch.cat(self.val_y, 0).to(torch.int)
        APs = self.metrics_ap(preds, y)
        for i in range(3):
            self.log(f'val_AP_bind{i}',APs[i], sync_dist=True)
        meanAP = self.metrics_map(preds, y)
        self.log('val_mAP', meanAP, prog_bar=True, sync_dist=True)
        self.val_preds.clear()
        self.val_y.clear()


    def test_step(self, batch, batch_idx):
        x, y = batch
        logits = self(x)
        loss = F.binary_cross_entropy_with_logits(logits, y)
        self.log('test_loss', loss)
        return loss


    def configure_optimizers(self):
        optimizer = torch.optim.AdamW(self.parameters(), lr=self.hparams.learning_rate, weight_decay=self.hparams.weight_decay)
        scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer)
        return [optimizer, ], [scheduler, ]
    def process_batch(self, batch):
        X, y = batch[0].clone().long(), batch[1].clone()
        return X, y


Writing models.py


In [7]:
%%writefile DALILmodels.py
from config import CFG
from models import LMTransformer, DemoModel

import tensorflow as tf
import torch

from nvidia.dali import pipeline_def, Pipeline
import nvidia.dali.fn as fn
import nvidia.dali.types as types
import nvidia.dali.tfrecord as tfrec
from nvidia.dali.plugin.pytorch import DALIGenericIterator, LastBatchPolicy



class DALILTransformer(LMTransformer):
    def __init__(self, *kargs, **kwargs):
        super().__init__(*kargs, **kwargs)

    def setup(self,stage=None):
        device_id = self.local_rank
        shard_id = self.global_rank
        num_shards = self.trainer.world_size

        train_pipe = belka_pipeline(
            batch_size=CFG.BATCH_SIZE,
            num_threads=4,
            device_id=device_id,
            device='cuda',
            shard_id=shard_id,
            num_shards=num_shards,
            paths=CFG.TRAINS,
            idxs=CFG.TRAIN_IDX,
            seed=CFG.SEED + 2 + device_id*2
        )
        valid_pipe = belka_pipeline(
            batch_size=CFG.BATCH_SIZE,
            num_threads=4,
            device_id=device_id,
            device='cuda',
            shard_id=shard_id,
            num_shards=num_shards,
            paths=CFG.VALIDS,
            idxs=CFG.VALID_IDX,
            seed=CFG.SEED-2
        )

        class LightningWrapper(DALIGenericIterator):
            def __init__(self, *kargs, **kwargs):
                super().__init__(*kargs, **kwargs)
            def __next__(self):
                out = super().__next__()
                out = out[0]
                return [out[k] for k in self.output_map]


        self.train_loader = LightningWrapper(train_pipe, ['X', 'y'],reader_name='Reader', last_batch_policy=LastBatchPolicy.DROP)
        self.valid_loader = LightningWrapper(valid_pipe, ['X', 'y'],reader_name='Reader', last_batch_policy=LastBatchPolicy.PARTIAL)

    def on_train_epoch_end(self):
        super().on_train_epoch_end()
        self.train_loader.reset()


    def train_dataloader(self):
        return self.train_loader

    def val_dataloader(self):
        return self.valid_loader
    
    def process_batch(self, batch):
        X, y = batch[0].clone().long(),  batch[1].clone()
        mask = (X != 0).clone().long()
        mask = torch.cat((torch.ones(X.shape[0],1).to(self.local_rank), mask), dim=1)
        
        return X, mask, y


class DALILDemoModel(DemoModel):
    def __init__(self, *kargs, **kwargs):
        super().__init__(*kargs, **kwargs)

    def setup(self,stage=None):
        device_id = self.local_rank
        shard_id = self.global_rank
        num_shards = self.trainer.world_size

        train_pipe = belka_pipeline(
            batch_size=CFG.BATCH_SIZE,
            num_threads=4,
            device_id=device_id,
            device='cuda',
            shard_id=shard_id,
            num_shards=num_shards,
            paths=CFG.TRAINS,
            idxs=CFG.TRAIN_IDX,
            seed=CFG.SEED + 2 + device_id*2
        )
        valid_pipe = belka_pipeline(
            batch_size=CFG.BATCH_SIZE,
            num_threads=4,
            device_id=device_id,
            device='cuda',
            shard_id=shard_id,
            num_shards=num_shards,
            paths=CFG.VALIDS,
            idxs=CFG.VALID_IDX,
            seed=1996
        )

        class LightningWrapper(DALIGenericIterator):
            def __init__(self, *kargs, **kwargs):
                super().__init__(*kargs, **kwargs)
            def __next__(self):
                out = super().__next__()
                out = out[0]
                return [out[k] for k in self.output_map]


        self.train_loader = LightningWrapper(train_pipe, ['X', 'y'],reader_name='Reader', last_batch_policy=LastBatchPolicy.DROP)
        self.valid_loader = LightningWrapper(valid_pipe, ['X', 'y'],reader_name='Reader', last_batch_policy=LastBatchPolicy.PARTIAL)

    def train_dataloader(self):
        return self.train_loader

    def val_dataloader(self):
        return self.valid_loader




@pipeline_def
def belka_pipeline(device, paths, idxs, seed,shard_id=0, num_shards=1, is_train=True):
    device_id = Pipeline.current().device_id

    inputs = fn.readers.tfrecord(
        path = paths,
        index_path = idxs,
        features={
            "x": tfrec.FixedLenFeature([CFG.SEQ_LENGTH], tfrec.int64, 0),
            "y": tfrec.FixedLenFeature([CFG.NUM_CLASSES], tfrec.float32, .0)
        },
        random_shuffle=is_train,
        num_shards=num_shards,
        shard_id=shard_id,
        initial_fill=CFG.BATCH_SIZE,
        seed=seed,
        name='Reader'
    )
    x = inputs['x']
    y = inputs['y']
    if device=='cuda':
        x = x.gpu()
        y = y.gpu()
    return x,y

Writing DALILmodels.py


In [8]:
%%writefile main.py
from config import CFG
from DALILmodels import DALILTransformer, belka_pipeline, DALILDemoModel

from logger.mylogger import get_my_logger

import os
import numpy as np
import pandas as pd
import glob
import random
import time
import tqdm

import torch
from torch import nn
import torch.nn.functional as F
from torch.utils.data import TensorDataset, DataLoader

import lightning as L
from lightning.pytorch.callbacks import EarlyStopping, ModelCheckpoint, LearningRateMonitor,TQDMProgressBar
from lightning.pytorch.loggers import CSVLogger, TensorBoardLogger
from torchmetrics.classification import MultilabelAveragePrecision as mAP


from nvidia.dali.plugin.pytorch import DALIGenericIterator, LastBatchPolicy


def set_seeds(seed):
    os.environ['PYTHONHASHSEED'] = str(seed)
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)

def set_logger(name):
    now = time.localtime()
    now = time.strftime("%Y-%m-%d-%H-%M-%S", now)
    log_name = f'{name}-{now}.log'
    logger = get_my_logger(CFG.BASE_DIR, log_name)
    return logger

def process_x(self, X):
    X = X.clone().long()
    mask = (X != 0).clone().long()
    mask = torch.cat((torch.ones(X.shape[0],1).to(0), mask), dim=1)

    return X, mask



def calc_validation_APS(model, model_name, mode, logger):
    logger.info('validation_APS start calucurate')
    valid_pipe = belka_pipeline(
            batch_size=CFG.BATCH_SIZE,
            num_threads=4,
            device_id=0,
            device='cuda',
            paths=CFG.VALIDS,
            idxs=CFG.VALID_IDX,
            seed=CFG.SEED-2,
            is_train=False
        )

    class LightningWrapper(DALIGenericIterator):
        def __init__(self, *kargs, **kwargs):
            super().__init__(*kargs, **kwargs)
        def __next__(self):
            out = super().__next__()
            out = out[0]
            return [out[k] for k in self.output_map]

    valid_loader = LightningWrapper(valid_pipe, ['X', 'y'],reader_name='Reader', last_batch_policy=LastBatchPolicy.PARTIAL)

    metrics_ap = mAP(3,thresholds=None, average='none')
    metrics_map = mAP(3, thresholds=None, average='micro')
    all_preds = []
    all_y = []
    model.to(0)
    model.eval()
    with torch.no_grad():
        for X, y in tqdm(valid_loader):
            X, mask = process_x(X)
            logits = model(X, mask)
            pred = F.sigmoid(logits)
            all_preds.append(pred)
            all_y.append(y)
    preds = torch.cat(all_preds, 0)
    y_eval = torch.cat(all_y, 0).to(torch.int)

    APs = metrics_ap(preds, y_eval)
    for i in range(3):
        logger.info(f'val_AP_bind{i} : {APs[i]}')
    meanAP = metrics_map(preds, y_eval)
    logger.info(f'valid_results: CV score = {meanAP}')
    preds = preds.clone().to('cpu').detach().numpy()
    val_results = pd.DataFrame({f'bind{i}': preds[:, i] for i in range(3)})
    val_results.to_csv(os.path.join(CFG.BASE_DIR, f'val_results_{model_name}_{mode}.csv'))
    logger.info(f'val_results write complite!\nfile_name: val_results_{model_name}_{mode}.csv')

def make_submit(model, model_name, mode, logger):
    logger.info('load test data')
    test_data = pd.read_parquet('/kaggle/input/belka-enc-dataset/test_enc.parquet')
    test_data = TensorDataset(torch.tensor(test_data.values, dtype=torch.int))
    test_loader= DataLoader(test_data, batch_size=CFG.BATCH_SIZE)
    logger.info('predict test data')
    test_preds=[]
    model.to(0)
    model.eval()
    with torch.no_grad():
        for (X,) in test_loader:
            X = X.to(0)
            X,mask = process_x(X)
            logits = model(X, mask)
            pred = F.sigmoid(logits)
            test_preds.append(pred)
    preds= torch.cat(test_preds, 0).to('cpu').detach().numpy()

    logger.info('writing submission.csv start')
    tst = pd.read_parquet('/kaggle/input/leash-BELKA/test.parquet')
    tst['binds'] = 0
    tst.loc[tst['protein_name']=='BRD4', 'binds'] = preds[(tst['protein_name']=='BRD4').values, 0]
    tst.loc[tst['protein_name']=='HSA', 'binds'] = preds[(tst['protein_name']=='HSA').values, 1]
    tst.loc[tst['protein_name']=='sEH', 'binds'] = preds[(tst['protein_name']=='sEH').values, 2]
    tst[['id', 'binds']].to_csv(f'submission.csv', index = False)
    logger.info('writing complete')



if __name__ == '__main__':
    logger = set_logger(CFG.MODEL_NAME)

    set_seeds(seed= CFG.SEED)
    logger.info(f'set seed: {CFG.SEED}')

    model_module = DALILTransformer()
#     model_module = DALILDemoModel()
#     model_module = torch.compile(model_module)
    logger.info(f'model has made.\n model:{model_module}')


    early_stop_callback = EarlyStopping(
        monitor= 'val_loss',
        mode= 'min',
        patience= 3,
        verbose= True
    )

#     loss_checkpoint_callback = ModelCheckpoint(
#         dirpath= f'{CFG.BASE_DIR}/models/',
#         filename= f'model-{CFG.MODEL_NAME}-{{epoch}}-{{val_loss:2f}}',
#         monitor= 'val_loss',
#         mode='min'
#         save_top_k= 1,
#         verbose= True,
#     )
    mAP_checkpoint_callback = ModelCheckpoint(
        dirpath= f'{CFG.BASE_DIR}/models/',
        filename= f'model-{CFG.MODEL_NAME}-{{epoch}}-{{val_mAP:2f}}',
        monitor= 'val_mAP',
        mode='max',
        save_top_k= 1,
        verbose= True,
    )

    progress_bar_callback = TQDMProgressBar(refresh_rate=1)

    lr_monitor = LearningRateMonitor(logging_interval='epoch')

    callbacks = [
        early_stop_callback,
#         loss_checkpoint_callback,
        mAP_checkpoint_callback,
        progress_bar_callback,
        lr_monitor,
    ]
    logger.info('set callbacks')

    ckpts = glob.glob(os.path.join('/kaggle/input/mamba-ep0/models', '*.ckpt'))
    ckpts.sort()
    latest_checkpoint = ckpts[-1] if len(ckpts) != 0 else None

    csvlogger = CSVLogger('/kaggle/working/lightning_logs','csv_logs'  )
    tblogger = TensorBoardLogger('/kaggle/working/lightning_logs','tb_logs')

    trainer = L.Trainer(
            max_epochs= CFG.EPOCHS,
#             max_time='00:08:30:00',
            callbacks= callbacks,
            accelerator= 'auto',
            enable_progress_bar= True,
            devices= 'auto',
            logger=[csvlogger, tblogger],
            strategy='ddp',
            precision="16-mixed"
        )
    logger.info('trainer has made')
    if latest_checkpoint:
        logger.info(f'load ckpt from {latest_checkpoint}')
    logger.info('training begin')
    trainer.fit(model_module, ckpt_path=latest_checkpoint)
    logger.info('training finish!')
#     model_module = DALILmamba.load_from_checkpoint(loss_checkpoint_callback.best_model_path)
#     calc_validation_APS(model_module, CFG.MODEL_NAME, 'loss', logger)
#     make_submit(model_module, CFG.MODEL_NAME, 'loss', logger)

    model_module = DALILTransformer.load_from_checkpoint(mAP_checkpoint_callback.best_model_path)
    calc_validation_APS(model_module, CFG.MODEL_NAME, 'mAP', logger)
    make_submit(model_module, CFG.MODEL_NAME, 'mAP', logger)

Writing main.py


In [9]:
# import numpy as np
# X = torch.randint(0,36,(128,142)).long()
# rng = np.random.default_rng(2024)
# mask = rng.integers(100,142, 128)
# for i, mk in enumerate(mask):
#     X[i][mk:] = 0
# y = torch.randint(0,2,(128,3))


In [10]:
# def process_batch(batch):
#     X, y = batch[0].clone().long(),  batch[1].clone()
#     mask = (X != 0).clone().long()
#     mask = torch.cat((torch.ones(X.shape[0],1), mask), dim=1)

#     return X, mask, y
# X, mask, y = process_batch((X,y))

In [11]:
# from config import *
# from models import RMSNorm, PositionalEncoder, Encoder
# import torch
# from torch import nn

# test = LMTransformer()
# test

In [12]:
# test(X,mask)

In [13]:
# mask

In [14]:
# mask = (batch != 0).long()
# mask = torch.cat((torch.ones(mask.shape[0],1), mask), dim=1)
# mask.shape

In [15]:
# mask.shape

In [16]:
# from models import RMSNorm, PositionalEncoder, Encoder
# import torch
# from torch import nn
# class test_model(nn.Module):
#     def __init__(self):
#         super().__init__()
#         self.norm = RMSNorm(dim=64)
#         self.embedding = nn.Embedding(num_embeddings= 37, 
#                                               embedding_dim= 64, 
#                                               padding_idx= 0)
#         self.cls_token = nn.Parameter(torch.zeros(1,1,64))
#         self.positional_encoder = PositionalEncoder(d_model= 64, 
#                                                     dropout_p= 0.2, 
#                                                     max_len = 128)
#         self.encoder = Encoder(hidden_dim= 64, 
#                                        num_heads= 2, 
#                                        num_layers= 1, 
#                                        dropout= 0.2)
#     def forward(self,x, mask):
#         x = embedding(x)
#         x = norm(x)
#         cls_tokens = cls_token.repeat_interleave(x.shape[0], dim= 0).clone()
#         #         cls_tokens = self.cls_token.clone().repeat(x.shape[0], 1, 1)
#         x = torch.cat((cls_tokens, x), dim= 1)
#         x = positional_encoder(x)
#         x = encoder(x, mask)
#         return x

In [17]:
# test = test_model()
# test.to(0)
# x = test(X,mask)
# x

In [18]:
!python main.py

/opt/conda/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()


2024-07-08 07:44:07.747753: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-08 07:44:07.747816: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-08 07:44:07.749314: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-07-08 07:44:10,963 Transformer-2024-07-08-07-44-10.log:27 get_my_logger [INFO]: logger has made. log_dir:/kaggle/working/logs
2024-07-08 07:44:10,965 Transformer-2024-07-08-07-44-10.log:132 <module> [INFO]: set seed: 2024
/opt/conda/lib/python3.10/site-packages/torch/nn/modules/transformer.py:282: UserWarning: enable_nested_tensor is True, but self.use_neste